In [1]:
import torch
import transformers

# Managing arrays
import numpy as np

# load the TensorBoard notebook extension
# %load_ext tensorboard

if torch.cuda.is_available():
  print("GPU is available.")
  device = torch.cuda.current_device()

else:
  print("Will work on CPU")


## IMPROVE THE MODEL

# # SOLUTION 1 (trivial): increase training epochs

# # SOLUTION 3: let's see what students can do !


GPU is available.


In [2]:
## DATA

from sklearn.datasets import fetch_20newsgroups

categories = [
    'comp.windows.x',
    'sci.med',
    'soc.religion.christian',
    'talk.politics.guns',
]

# download data if not already present in data_home
trainset = fetch_20newsgroups(subset='train', categories=categories, shuffle=True, random_state=42, data_home='./scikit_learn_data')
testset = fetch_20newsgroups(subset='test', categories=categories, shuffle=True, random_state=42, data_home='./scikit_learn_data')

# define input data and labels for training and testing
x_train = trainset.data
y_train = trainset.target
x_test = testset.data
y_test = testset.target

In [3]:

# # SOLUTION (yes, we are cool)
print('taille du jeu de données: \n \t {} posts de forum en total'.format(len(x_train) + len(x_test)))
print('\t {} posts pour le training'.format(len(x_train)))
for i in range(len(categories)):
    num = sum(y_train == i)
    print("\t\t {} {}".format(num, categories[i]))
    print('\t {} posts pour le test'.format(len(x_test)))
for i in range(len(categories)):
    num = sum(y_test == i)
    print("\t\t {} {}".format(num, categories[i]))

print('\n')
print('EXEMPLE: \n')
print(x_train[0])


def clean_post(post: str, remove_start: tuple):
    clean_lines = []
    for line in post.splitlines():
        if not line.startswith(remove_start):
            clean_lines.append(line)
    return '\n'.join(clean_lines)


taille du jeu de données: 
 	 3885 posts de forum en total
	 2332 posts pour le training
		 593 comp.windows.x
	 1553 posts pour le test
		 594 sci.med
	 1553 posts pour le test
		 599 soc.religion.christian
	 1553 posts pour le test
		 546 talk.politics.guns
	 1553 posts pour le test
		 395 comp.windows.x
		 396 sci.med
		 398 soc.religion.christian
		 364 talk.politics.guns


EXEMPLE: 

From: gilham@csl.sri.com (Fred Gilham)
Subject: Poem
Organization: Computer Science Lab, SRI International, Menlo Park, CA.
Lines: 145


          The Sophomore
          (Romans 1:22)

The sophomore says, ``What is truth?''
and turns to bask in the admiration of his peers.

How modern how daring how liberating
How modern how daring how liberating
they chant

The sophomore, being American
Doesn't know
That his ``question''

   modern
       skeptical
           cynical

Was asked before, by a

   modern
       skeptical
           cynical
   urbane cosmopolitan

Politician (appointed not elected)
Who 

In [4]:

# SOLUTION (yes, again, we are cool)
remove_start = (
    'From:',
    'Subject:',
    'Reply-To:',
    'In-Reply-To:',
    'Nntp-Posting-Host:',
    'Organization:',
    'X-Mailer:',
    'In article <',
    'Lines:',
    'NNTP-Posting-Host:',
    'Summary:',
    'Article-I.D.:'
)
x_train = [clean_post(p, remove_start) for p in x_train]
x_test = [clean_post(p, remove_start) for p in x_test]



In [5]:
## TOKENISATION


from torch.utils.data import Dataset, DataLoader
from transformers import DistilBertTokenizer
from transformers import DistilBertTokenizerFast

MAX_LEN = 512

tokenizer = DistilBertTokenizerFast.from_pretrained('distilbert-base-uncased', padding=True, truncation=True)

# let's check out how the tokenizer works
for n in range(3):
    # tokenize forum post
    tokenizer_out = tokenizer(x_train[n])
    # convert numerical tokens to alphabetical tokens
    encoded_tok = tokenizer.convert_ids_to_tokens(tokenizer_out.input_ids)
    # decode tokens back to string
    decoded = tokenizer.decode(tokenizer_out.input_ids)
    print(tokenizer_out)
    print(encoded_tok, '\n')
    print(decoded, '\n')
    print('---------------- \n')


class PostsDataset(Dataset):
    def __init__(self, posts, labels, tokenizer, max_len):
        # variables that are set when the class is instantiated
        self.posts = posts
        self.labels = labels
        self.tokenizer = tokenizer
        self.max_len = max_len

    def __len__(self):
        return len(self.posts)

    def __getitem__(self, item):
        # select the post and its category
        post = str(self.posts[item])
        label = self.labels[item]
        # tokenize the post
        tokenizer_out = self.tokenizer(
            post,
            add_special_tokens=True,
            max_length=self.max_len,
            return_token_type_ids=False,
            pad_to_max_length=True,
            return_attention_mask=True,
            return_tensors='pt',
            truncation=True
        )
        # return a dictionary with the output of the tokenizer and the label
        return  {
            'input_ids': tokenizer_out['input_ids'].flatten(),
            'attention_mask': tokenizer_out['attention_mask'].flatten(),
            'label': torch.tensor(label, dtype=torch.long)
        }



Token indices sequence length is longer than the specified maximum sequence length for this model (744 > 512). Running this sequence through the model will result in indexing errors


{'input_ids': [101, 1996, 13758, 1006, 10900, 1015, 1024, 2570, 1007, 1996, 13758, 2758, 1010, 1036, 1036, 2054, 2003, 3606, 1029, 1005, 1005, 1998, 4332, 2000, 19021, 2243, 1999, 1996, 17005, 1997, 2010, 12746, 1012, 2129, 2715, 2129, 15236, 2129, 5622, 5677, 5844, 2129, 2715, 2129, 15236, 2129, 5622, 5677, 5844, 2027, 16883, 1996, 13758, 1010, 2108, 2137, 2987, 1005, 1056, 2113, 2008, 2010, 1036, 1036, 3160, 1005, 1005, 2715, 18386, 26881, 2001, 2356, 2077, 1010, 2011, 1037, 2715, 18386, 26881, 3923, 2063, 24686, 3761, 1006, 2805, 2025, 2700, 1007, 2040, 3047, 2000, 2444, 2048, 4595, 2086, 3283, 1012, 2066, 2116, 8801, 2002, 8725, 2625, 2055, 15084, 2084, 3463, 2625, 2055, 4515, 2084, 2965, 2625, 2055, 2505, 2084, 4363, 2010, 3105, 1006, 1998, 2010, 2132, 1007, 1012, 2057, 2453, 2655, 2032, 1037, 2978, 12077, 2295, 1036, 3813, 1005, 2052, 2022, 2785, 2121, 1006, 1998, 2053, 4797, 13125, 1010, 2040, 2292, 6343, 2175, 1010, 4191, 2012, 2010, 27327, 2791, 1007, 2002, 2134, 1005, 1056, 2

In [6]:

# instantiate two PostsDatasets
train_dataset = PostsDataset(x_train, y_train, tokenizer, MAX_LEN)
test_dataset = PostsDataset(x_test, y_test, tokenizer, MAX_LEN)


In [7]:
## MODEL

from transformers import DistilBertModel

from transformers import DistilBertPreTrainedModel, DistilBertConfig

PRE_TRAINED_MODEL_NAME = 'distilbert-base-uncased'


distilbert = DistilBertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)

first_post = train_dataset[0]

hidden_state = distilbert(
    input_ids=first_post['input_ids'].unsqueeze(0), attention_mask=first_post['attention_mask'].unsqueeze(0)
)

print(hidden_state[0].shape)

print(distilbert.config)



class Pre_Classifier(torch.nn.Module):
    def __init__(self, config):
        super().__init__()
        self.pre_classifier = torch.nn.Linear(config.dim, config.dim)
        self.activation = torch.nn.Tanh()

    def forward(self, hidden_states):
        first_token_tensor = hidden_states[:, 0]
        pooled_output = self.pre_classifier(first_token_tensor)
        pooled_output = self.activation(pooled_output)
        return pooled_output

class DistilBertForPostClassification(DistilBertPreTrainedModel):
    def __init__(self, config, num_labels, freeze_encoder=False):
        # instantiate the parent class DistilBertPreTrainedModel
        super().__init__(config)
        # instantiate num. of classes
        self.num_labels = num_labels
        # instantiate and load a pretrained DistilBERT model as encoder
        self.encoder = DistilBertModel.from_pretrained(PRE_TRAINED_MODEL_NAME)
        # freeze the encoder parameters if required (Q1)
        if freeze_encoder:
            for param in self.encoder.parameters():
                param.requires_grad = False
        #Adding a pre classifier to improve the classifier efficiency
        self.pre_classifier = Pre_Classifier(config)
        # the classifier: a feed-forward layer attached to the encoder's head
        
        self.classifier = torch.nn.Linear(in_features=config.dim, out_features=self.num_labels, bias=True)
        
        # instantiate a dropout function for the classifier's input
        self.dropout = torch.nn.Dropout(p=0.3)


    def forward(
            self,
            input_ids=None,
            attention_mask=None,
            head_mask=None,
            inputs_embeds=None,
            labels=None,
            output_attentions=None,
            output_hidden_states=None,
    ):
        # encode a batch of sequences with DistilBERT
        encoder_output = self.encoder(
            input_ids=input_ids,
            attention_mask=attention_mask,
            head_mask=head_mask,
            inputs_embeds=inputs_embeds,
            output_attentions=output_attentions,
            output_hidden_states=output_hidden_states,
        )
        # extract the hidden representations from the encoder output
        hidden_state = encoder_output[0]  # (bs, seq_len, dim)
        # only select the encoding corresponding to the first token
        # of each sequence in the batch (Q2)
        pooled_output = hidden_state[:, 0]  # (bs, dim)

        pooled_output = self.pre_classifier(hidden_state)
        # apply dropout
        pooled_output = self.dropout(pooled_output)  # (bs, dim)

        # feed into the classifier
        logits = self.classifier(pooled_output)  # (bs, dim)

        outputs = (logits,) + encoder_output[1:]



        if labels is not None: # (Q3)
            # instantiate loss function
            loss_fct = torch.nn.CrossEntropyLoss()
            # calculate loss
            loss = loss_fct(logits.view(-1, self.num_labels), labels.view(-1))
            # aggregate outputs
            outputs = (loss,) + outputs

        return outputs  # (loss), logits, (hidden_states), (attentions)




Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
D:\anaconda\lib\site-packages\transformers\tokenization_utils_base.py:2221: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or 

torch.Size([1, 512, 768])
DistilBertConfig {
  "_name_or_path": "distilbert-base-uncased",
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.14.1",
  "vocab_size": 30522
}



In [8]:
## TRAINING

from transformers import Trainer, TrainingArguments
from sklearn.metrics import precision_recall_fscore_support, accuracy_score

SOLUTION = 1

# instantiate model




if SOLUTION == 1:
    model = DistilBertForPostClassification(config=distilbert.config, num_labels=len(categories), freeze_encoder = True)
else:
    model = DistilBertForPostClassification(config=distilbert.config, num_labels=len(categories), freeze_encoder = True)

# print info about model's parameters
total_params = sum(p.numel() for p in model.parameters())
model_parameters = filter(lambda p: p.requires_grad, model.parameters())
trainable_params = sum([np.prod(p.size()) for p in model_parameters])
    
def compute_metrics(pred):
    labels = pred.label_ids
    preds = pred.predictions.argmax(-1)
    precision, recall, f1, _ = precision_recall_fscore_support(labels, preds)
    acc = accuracy_score(labels, preds)
    return {
        'accuracy': acc,
        'f1': f1,
        'precision': precision,
        'recall': recall
    }


lr = 0.00005
training_args = TrainingArguments(
    output_dir='./results',          
    logging_dir='./logs',
    logging_first_step=True,
    logging_steps=50,
    num_train_epochs=4,              
    per_device_train_batch_size=8,  
    learning_rate=lr,
    weight_decay=0.01        
)

trainer = Trainer(
    model=model,                         
    args=training_args,                  
    train_dataset=train_dataset,         
    compute_metrics=compute_metrics
)

train_results = trainer.train()

test_results = trainer.predict(test_dataset=test_dataset)

def round_res(ml):
    return [ round(elem,5) for elem in ml ]

Accuracy= round(test_results.metrics['test_accuracy'], 5)
Precision= round_res(test_results.metrics['test_precision'])
Recall= round_res(test_results.metrics['test_recall'])
F1 = round_res(test_results.metrics['test_f1'])

print('Predictions: \n', test_results.predictions)
print('\nAccuracy: ', Accuracy)
print('Precision: ', Precision)
print('Recall: ',Recall)
print('F1: ', F1)
print(categories)
MODEL_PATH = './my_model'
trainer.save_model(MODEL_PATH)



Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.bias', 'vocab_transform.bias', 'vocab_transform.weight', 'vocab_layer_norm.weight', 'vocab_projector.bias', 'vocab_projector.weight']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
***** Running training *****
  Num examples = 2332
  Num Epochs = 4
  Instantaneous batch size per device = 8
  Total train batch size (w. parallel, distributed & accumulation) = 8
  Gradient Accumulation steps = 1
  Total optimization steps = 1168


Step,Training Loss
1,1.285800
50,1.288100
100,1.104600
150,0.922300
200,0.771700
250,0.698100
300,0.578000
350,0.526400
400,0.431800
450,0.440500


Saving model checkpoint to ./results\checkpoint-500
Configuration saved in ./results\checkpoint-500\config.json
Model weights saved in ./results\checkpoint-500\pytorch_model.bin
D:\anaconda\lib\site-packages\transformers\tokenization_utils_base.py:2221: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(
Saving model checkpoint to ./results\checkpoint-1000
Configuration saved in ./results\checkpoint-1000\config.json
Model weights saved in ./results\checkpoint-1000\pytorch_model.bin
D:\anaconda\lib\site-packages\transformers\tokenization_utils_base.py:2221: FutureWarning: The `pad_to_max_length` argument

Saving model checkpoint to ./my_model
Configuration saved in ./my_model\config.json


Predictions: 
 [[ 6.0417085  -0.94372606 -2.5727444  -1.865699  ]
 [ 3.8881085  -0.22453788 -1.9770339  -1.1293151 ]
 [ 1.6429119  -0.21760496 -0.7635118  -0.3701588 ]
 ...
 [-0.32982612  0.29814097 -0.48656666  0.7520254 ]
 [-0.2223302   0.0083893  -1.2901748   1.7011836 ]
 [-2.4530582   0.9094646   2.0562832  -0.54099405]]

Accuracy:  0.94269
Precision:  [0.9525, 0.96216, 0.93612, 0.92021]
Recall:  [0.96456, 0.89899, 0.95729, 0.95055]
F1:  [0.95849, 0.9295, 0.94658, 0.93514]
['comp.windows.x', 'sci.med', 'soc.religion.christian', 'talk.politics.guns']


Model weights saved in ./my_model\pytorch_model.bin


In [9]:
MODEL_PATH = './my_model'
trainer.save_model(MODEL_PATH)

Saving model checkpoint to ./my_model
Configuration saved in ./my_model\config.json
Model weights saved in ./my_model\pytorch_model.bin


In [10]:

## PREDICTIONS

model = DistilBertForPostClassification.from_pretrained(
    './my_model', config=distilbert.config, num_labels=len(categories)).to(device)
for sentence in ['Lung cancer is a deadly disease.', 'God is love', 'How can you install Microsoft Office extensions?', 'Gun killings increase every year.']:
    encoding = tokenizer.encode_plus(sentence)
    encoding['input_ids'] = torch.tensor([encoding.input_ids]).to(device)
    encoding['attention_mask'] = torch.tensor(encoding.attention_mask).to(device)
    out = model(**encoding)
    categories_probability = torch.nn.functional.softmax(out[0], dim=1).flatten()
    print(sentence)
    print('\tProbabilities assigned by the model : ')
    for n,c in enumerate(categories):
        print('\t\t{} : {}'.format(c, categories_probability[n]))
    print('\n\t--> Prediction :', categories[categories_probability.argmax()])
    print('------------------------------------------------\n')



loading weights file ./my_model\pytorch_model.bin
loading configuration file https://huggingface.co/distilbert-base-uncased/resolve/main/config.json from cache at C:\Users\Laran/.cache\huggingface\transformers\23454919702d26495337f3da04d1655c7ee010d5ec9d77bdb9e399e00302c0a1.91b885ab15d631bf9cee9dc9d25ece0afd932f2f5130eba28f2055b2220c0333
Model config DistilBertConfig {
  "activation": "gelu",
  "architectures": [
    "DistilBertForMaskedLM"
  ],
  "attention_dropout": 0.1,
  "dim": 768,
  "dropout": 0.1,
  "hidden_dim": 3072,
  "initializer_range": 0.02,
  "max_position_embeddings": 512,
  "model_type": "distilbert",
  "n_heads": 12,
  "n_layers": 6,
  "pad_token_id": 0,
  "qa_dropout": 0.1,
  "seq_classif_dropout": 0.2,
  "sinusoidal_pos_embds": false,
  "tie_weights_": true,
  "transformers_version": "4.14.1",
  "vocab_size": 30522
}

loading weights file https://huggingface.co/distilbert-base-uncased/resolve/main/pytorch_model.bin from cache at C:\Users\Laran/.cache\huggingface\tran

Lung cancer is a deadly disease.
	Probabilities assigned by the model : 
		comp.windows.x : 0.0011246519861742854
		sci.med : 0.9846445322036743
		soc.religion.christian : 0.006960222031921148
		talk.politics.guns : 0.007270723581314087

	--> Prediction : sci.med
------------------------------------------------

God is love
	Probabilities assigned by the model : 
		comp.windows.x : 0.008614864200353622
		sci.med : 0.05478717386722565
		soc.religion.christian : 0.9028907418251038
		talk.politics.guns : 0.03370722755789757

	--> Prediction : soc.religion.christian
------------------------------------------------

How can you install Microsoft Office extensions?
	Probabilities assigned by the model : 
		comp.windows.x : 0.9807853698730469
		sci.med : 0.012270537205040455
		soc.religion.christian : 0.0029387103859335184
		talk.politics.guns : 0.004005391150712967

	--> Prediction : comp.windows.x
------------------------------------------------

Gun killings increase every year.
	Probabili

Intro)

Here the results of all our experiment.
The initial state is computing without fine tuning and 4 epoch,
here the result of this initial state.

\begin{tabular}{| T | V |}
Accuracy & 0.90663 \\
Precision & [0.87244, 0.90556, 0.91525, 0.94135] \\
Recall & [0.96962, 0.82323, 0.94975, 0.88187]  \\
F1 & [0.91847, 0.86243, 0.93218, 0.91064] \\
\end{tabular}

Now let's talk about the configuration of my computers.

The tests were made on my GPU, who achieve a time of approximately 100 seconds for this basic example, instead of 7500 with my CPU.

Let's now see what are the results of our experiments.

1) Increase the numbers of epoch 

As said previously, the fact that i can computes with my GPU, help a lot for the computation times. \\
Thanks to that, we can try multiples numbers of epoch.\\
First we chose to increase just by one to see if there is a big change.\\
Then we will follow the fibonacci principle and increase it by 2 and 3, to increase gradually the numbers of epoch. \\
To resume, for the following example, we take as epoch 5, 7 and 10.
\begin{tabular}{ | T|C|S|D| }
Epoch & 5 & 7 & 10 \\
Accuracy & 0.9207 & 0.9311 & 0.9368 \\
Precision & [0.9021, 0.9160, 0.9242, 0.9461] & [0.92289, 0.93048, 0.92944, 0.94334] & [0.9407, 0.9513, 0.9353, 0.9202] \\
Recall & [0.9797, 0.8813, 0.9497, 0.8681] &[0.96962, 0.87879, 0.9598, 0.91484] & [0.9645, 0.8888, 0.9447, 0.9505] \\
F1 & [0.9393, 0.8983, 0.9368, 0.9054] & [0.94568, 0.9039, 0.94438, 0.92887] &[0.9525, 0.9191, 0.94, 0.9351] \\
\end{tabular}

We can see that when we increase the numbers of steps, the accuracy will increase too, our result are up to 1.7% more accurate.

The biggest improvement were made between four and five epoch, where the accuracy is increase by 1.09%.
But as we can see, between five epoch and ten epoch the values of the accuracy and the F1-mesure start to converge.


2) Fine tune the model 

This a good starting point, now let's see what append when we use fine tuning, we can estimate that the model will be more efficient, because all the layers will be trained.

\begin{tabular}{ | T | C | }
Accuracy & 0.9672 \\
Precision & [0.9677, 0.9839, 0.9668, 0.9751]\\
Recall & [0.9848, 0.9242, 0.98994, 0.9698]\\
F1 & [0.9762, 0.9531, 0.9669, 0.9054] \\
\end{tabular}

We can see that there is a big improvement, but that the running time is more longer, this can be explain by the fact that the back propagation will be applied in all the layer. But for the F1-mesure we can see that some values are lower than the initial case. The model seems overfit.

3)
We tried to make a lot of improvement.
The first idea was to combine a large number of epoch and the fine tuning, but we saw that the accurate quickly converge to 0.97, and this is not improving the model.

After this failure we chose to keep our initial state, so from now all the following experiment have been done with 4 epoch and no fine tuning. 
We will show you only the experiment who reach to an accuracy greater than 94%, we found the others non relevant since the initial case with 10 epoch almost reach it.  

Our second idea was to adjusts the learning rate.
We saw that when we are increasing the number of epoch the model tend to an higher value, the goal here is just to increase the learning rate to have good result even if the number of epoch is small.
After doing a lot of experiment we chose to keep a learning rate of 0.005.
Let's see our new result.

\begin{tabular}{ | T | C | }
Accuracy &  0.950428 \\
Precision &  [0.96222, 0.97035, 0.92399, 0.9478] \\
Recall & [0.96709, 0.90909, 0.97739, 0.9478] \\
F1 &  [0.96465, 0.93872, 0.94994, 0.9478] \\
\end{tabular}

Our final idea was to add a layer in the model before the classifier, we saw that there is a lot of variance in the observation. The goal here is to apply a mask to normalize the data. To do it we made a pre classifier with only one layer of size input x input.  
Let's see the result of this experiment


\begin{tabular}{ | T | C | }
Accuracy &  0.94527 \\
Precision &  [0.96438, 0.96467, 0.91335, 0.94247] \\
Recall &  [0.95949, 0.89646, 0.9799, 0.94505] \\
F1 & [0.96193, 0.92932, 0.94545, 0.94376] \\
\end{tabular}

As we can see that precision values have less variance, that was what we want, but the accuracy is lower than expected .

4) Conclusion

Now we want to make a point on what we learn during this pratical.

First of all we learn the pratical part of building a machine learning architecture, and all the steps we have to do, how to change the numbers of steps, the learning rate ... 

Secondly we saw how to Fine tune a model and how really is the improvement. 

To finish we also learned a lot of things when we try made the classification layers. 

Our first idea was to make a lot of hidden layers and we saw that the results was worst the basic result.

So we think about mixing fine tuning and a lot of layers classification, to improve this result, it was clearly not significant.

And to finish when we saw that a simple layer is enought to grant great performance, we conclude that so the performance will not depend of numbers of layers.

This is the most important point we learned in this pratical, even with a high number of steaps, increasing the numbers of layer is not always a good solution, and can be just negative for the architecture.




